In [1]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/CNN/ext_model/ext_model.ipynb'

In [2]:
%run {RUN_DIR}

In [3]:
class RnnBasicModel(CnnExtModel):
    pass

In [5]:
def rnn_basic_alloc_rnn_layer(self, input_shape, hconfig):
    
    inseq = get_conf_param(hconfig, 'inseq', True)
    outseq = get_conf_param(hconfig, 'outseq', True)
    
    if inseq:
        timesteps1, timefeats = inpu_shape
    else:
        timesteps1 = get_conf_param(hconfig, 'timesteps') + 1
        timefeats = np.prod(input_shape)
        
    recur_size = get_conf_param(hconfig, 'recur_size')
    
    ex_inp_dim = timefeats + recur_size
    weight, bias = self.alloc_param_pair([ex_inp_dim, recur_size])
    
    if outseq:
        output_shape = [timesteps1, recur_size]
        
    else:
        
        output_shape = [recur_size]
    rnn_info = [inseq, outseq, timesteps1, timefeats, recur_size]
    
    return {'w':weight, 'b': bias, 'info':rnn_info}, output_shape


RnnBasicModel.alloc_rnn_layer = rnn_basic_alloc_rnn_layer

In [6]:
def rnn_basic_forward_rnn_layer(self, x, hconfig, pm):
    inseq, outseq, timesteps1, timefeats, recur_size = pm['info']
    mb_size = x.shape[0]
    
    if inseq:
        x_slices = x[:,1:,:].transpose([1,0,2])
        lengths = x[:,0,0].astype(np.int32)
        timesteps = np.max(lengths)
        
    else:
        
        x_slice = x
        timesteps = timesteps1 - 1
        lengths = [timesteps] * mb_size
        
    recurrent = np.zeros([mb_size, recur_size])
    
    output, aux_steps = [],[]
    
    for n in range(timesteps):
        if inseq: x_slice = x_slices[n]
        ex_inp = np.hstack([x_slice, recurrent])
        affine = np.matmul(ex_inp, pm['w']) + pm['b']
        recurrent = self.activate(affine, hconfig)
        
        outputs.append(recurrent)
        aux_steps.append(ex_inp)
        
    if outseq:
        
        output = np.zeros([mb_size, timesteps1, recur_size])
        output[:, 0, 0] = lengths
        output[:,1:,:] = np.asarray(outputs).transpose([1,0,2])
        
    else:
        
        output = np.zeros([mb_size, recur_size])
        for n in range(mb_size):
            output[n] = outputs[lengths[n]-1[n]]
            
    return output, [x, lengths, timesteps, outputs, aux_steps]


RnnBasicModel.forward_rnn_layer = rnn_basic_forward_rnn_layer